In [1]:
import pandas as pd
import mat73
from datetime import datetime
import numpy as np
from tqdm import tqdm

In [2]:
df_brandon = pd.read_csv('work_tables/EEGs_And_Reports_20231024.csv')

In [3]:
df = pd.read_excel('tables/batch1.xlsx')
df = df.rename({'file':'HashFolderName'},axis=1)
df=df.merge(df_brandon[['HashFolderName','SiteID','SessionID_new','EEGFolder']],on='HashFolderName')

In [4]:
# root = 'bdsp/opendata/EEG/data/'
# i= 0
# sub_df = df.iloc[i].copy()
# path = f'{sub_df.SiteID}/{sub_df.EEGFolder}/{sub_df.HashFolderName}/'
# filename = '_'.join([sub_df.HashFolderName.split('_')[0],'0']+sub_df.HashFolderName.split('_')[1:3])+'.mat'

In [5]:
# root = 'bdsp/opendata/EEG/data/'
# i= 0
# x = df.iloc[i].copy()
# path = f'{sub_df.SiteID}/{sub_df.EEGFolder}/{sub_df.HashFolderName}/'
# filename = '_'.join([sub_df.HashFolderName.split('_')[0],'0']+sub_df.HashFolderName.split('_')[1:3])+'.mat'

# signal = mat73.loadmat(root+path+filename)
# data=signal['data']
# Fs=signal['Fs']

In [6]:
# Combine date and time into a datetime object
def get_time_from_filename(filename):
    # filename in format blabla_y,m,d_h,m,s.mat
    # returns timestamp obect
    date_str = filename.replace('.mat','').split('_')[-2]
    time_str = filename.replace('.mat','').split('_')[-1]

    date_object = datetime.strptime(date_str, '%Y%m%d')
    time_object = datetime.strptime(time_str, '%H%M%S').time()

    # Combine date and time into a datetime object
    start = datetime.combine(date_object.date(), time_object)   
    return start

In [22]:
def get_event_snippet(data,absolute_eeg_start_time,absolute_event_time,windowsize,all_channels,desired_channels):
    # takes a piece of signal of shape [channel, ts], the timestamp of the event and the windowsize
    # returns the event pm windowsize, shape [channel, windowsize]
    relative_event_time = int((absolute_event_time-absolute_start_time).total_seconds())
    snippet_start = int((relative_event_time-windowsize/2)*Fs)
    snippet_end = int((relative_event_time+windowsize/2)*Fs)
    desired_channel_ids = [all_channels.index(channel) for channel in desired_channels]
    return data[desired_channel_ids,snippet_start:snippet_end]

In [23]:
# process dataframe
root = 'bdsp/opendata/EEG/data/'
HashFolderNames = df.HashFolderName.unique().tolist()


In [24]:
def generate_path_and_filename(df):
    SiteID,EEGFolder,HashFolderName = df.iloc[0][['SiteID','EEGFolder','HashFolderName']].to_list()

    # generate path to file and filename
    path = SiteID+'/'+EEGFolder+'/'+HashFolderName+'/'
    # insert 0 to create Hashfoldername_0_date_time
    filename = '_'.join([HashFolderName.split('_')[0],'0']+HashFolderName.split('_')[1:3])+'.mat'


In [29]:
def process_EEG_file(SiteID,EEGFolder,HashFolderName):
    # generate path to file and filename
    path = SiteID+'/'+EEGFolder+'/'+HashFolderName+'/'
    # insert 0 to create Hashfoldername_0_date_time
    filename = '_'.join([HashFolderName.split('_')[0],'0']+HashFolderName.split('_')[1:3])+'.mat'

    # load signal and data
    signal = mat73.loadmat(root+path+filename)
    data=signal['data']
    Fs=signal['Fs']
    channels=signal['channels']
    absolute_start_time = get_time_from_filename(filename)

    return signal,data,Fs,channels,absolute_start_time

In [31]:
# process EEG file
i = 100
sub_df = df[df.HashFolderName==HashFolderNames[i]]
SiteID,EEGFolder,HashFolderName = sub_df.iloc[0][['SiteID','EEGFolder','HashFolderName']].to_list()
signal,data,Fs,channels,absolute_eeg_start_time = process_EEG_file(SiteID,EEGFolder,HashFolderName)


In [32]:
def load_event(data,absolute_eeg_start_time,absolute_event_time,windowsize):
    relative_event_time = int((absolute_event_time-absolute_start_time).total_seconds())
windowsize = 15
snippet = get_event_snippet(data=data,event_time=relative_event_time,windowsize=windowsize,all_channels=channels,desired_channels=desired_channels)


SyntaxError: invalid syntax (1042894638.py, line 1)

In [28]:
# load event
j = 0
absolute_event_time = sub_df.iloc[j].time
